In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, InvertibleMapper, StaticCovariatesTransformer
from darts.dataprocessing.transformers.missing_values_filler import MissingValuesFiller
from darts.metrics import rmsle
from sklearn.metrics import mean_squared_log_error
from darts.models import LinearRegressionModel, LightGBMModel, XGBModel, CatBoostModel
from sklearn.preprocessing import OneHotEncoder
from tqdm.notebook import tqdm_notebook

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [36]:
pd.set_option('display.max_columns', None)

In [37]:
data=pd.read_csv(r'novi_datasetovi\train_test_final.csv',parse_dates=['date'])
data.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,lag_16_sales,lag_17_sales,lag_18_sales,lag_19_sales,lag_20_sales,lag_30_sales,lag_365_sales,lag_730_sales,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,date_index,season,workday,wageday,day_to_nearest_holiday,day_from_nearest_holiday,lag_1_oil,lag_2_oil,lag_3_oil,lag_4_oil
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN


In [38]:
def get_pipeline(static_covs_transform=False,log_transform=False):
    lst = []
    
    filler = MissingValuesFiller(n_jobs=-1)
    lst.append(filler)
    
    if static_covs_transform:
        static_covs_transformer = StaticCovariatesTransformer(
            transformer_cat=OneHotEncoder(), #one hot encoding static covariates
            n_jobs=-1,
        )
        lst.append(static_covs_transformer)

    # perform log transformation on sales
    if log_transform:
        log_transformer = InvertibleMapper(
            fn=np.log1p,
            inverse_fn=np.expm1,
            n_jobs=-1,
        )
        lst.append(log_transformer)

    # rescale time series
    scaler = Scaler()
    lst.append(scaler)

    pipeline = Pipeline(lst)
    return pipeline

In [39]:
def get_target_series(static_cols,log_transform=True):    
    target_dict = {} #key is family, value is array of time series of sales by stores
    pipe_dict = {} #key is family, value is pipeline
    id_dict = {} #key is family, value is pair of store and family

    for fam in tqdm_notebook(data.family.unique(), desc="Extracting target series"):

        #using train and splitting by family
        df = data[(data.family.eq(fam)) & (data.date.le('2017-08-15'))] 
        
        # initialize transformation pipeline for target series
        pipe = get_pipeline(True, log_transform=log_transform)
        
        # extract target series together with static covariates
        target = TimeSeries.from_group_dataframe(
            df=df,
            time_col="date",
            value_cols="sales",
            group_cols="store_nbr",
            static_cols=static_cols,
        )
        
        # record identity of each target series
        target_id = [{"store_nbr": t.static_covariates.store_nbr, "family": fam} #pair family store
                     for t in target]
        id_dict[fam] = target_id

        # apply transformations
        target = pipe.fit_transform(target)
        target_dict[fam] = [t.astype(np.float32) for t in target]

        pipe_dict[fam] = pipe[2:]  #without MissingValuesFiller and OHEnc
        
    return target_dict, pipe_dict, id_dict

In [40]:
static_cols = ["city", "state", "type", "cluster"]

target_dict, pipe_dict, id_dict = get_target_series(static_cols)

Extracting target series:   0%|          | 0/33 [00:00<?, ?it/s]

In [41]:
def get_covariates(
    past_cols,
    future_cols,
):
    past_dict = {} #key is family, value is array of time series of transactions by stores
    future_dict = {} #key is family, value is array of array of time series of future covariate by stores for all future covariates
    
    # initialize transformation pipeline for covariates
    covs_pipe = get_pipeline()

    for fam in tqdm_notebook(data.family.unique(), desc="Extracting covariates"):
        # filter data for each model
        df = data[data.family.eq(fam)]
        
        # extract past covariates
        past_covs = TimeSeries.from_group_dataframe(
            df=df[df.date.le('2017-08-15')],
            time_col="date",
            value_cols=past_cols,
            group_cols="store_nbr",
        )
        past_covs = [p.with_static_covariates(None) for p in past_covs]
        past_covs = covs_pipe.fit_transform(past_covs)
        
        past_dict[fam] = [p.astype(np.float32) for p in past_covs]

        # extract future covariates
        future_covs = TimeSeries.from_group_dataframe(
            df=df,
            time_col="date",
            value_cols=future_cols,
            group_cols="store_nbr",
        )
        future_covs = [f.with_static_covariates(None) for f in future_covs]
        future_covs = covs_pipe.fit_transform(future_covs)
        
        future_dict[fam] = [f.astype(np.float32) for f in future_covs]
            
    return past_dict, future_dict

In [42]:
# past covariates
past_cols = ["transactions"]

# future covariates
future_cols = [
    "work_day", "N Batalla de Pichincha", "N Carnaval", "N Cyber Monday", "N Dia de Difuntos", "N Dia de la Madre", "N Dia del Trabajo", "N Futbol", "N Independencia de Cuenca", "N Independencia de Guayaquil", "N Navidad", "N Primer dia del ano", "N Terremoto Manabi", "N Viernes Santo", "oil_price", "transactions", "month", "day_of_month", "day_of_year", "week_of_month", "week_of_year", "day_of_week", "year", "is_wknd", "quarter", "is_month_start", "is_month_end", "is_quarter_start", "is_quarter_end", "is_year_start", "is_year_end", "date_index", "season", "workday", "wageday", "day_to_nearest_holiday", "day_from_nearest_holiday"]
past_dict, future_dict = get_covariates(past_cols, future_cols)

Extracting covariates:   0%|          | 0/33 [00:00<?, ?it/s]

In [43]:
TRAINER_CONFIG = {

    "target_dict": target_dict,
    "pipe_dict": pipe_dict,
    "id_dict": id_dict,
    "past_dict": past_dict,
    "future_dict": future_dict,
    
    # time series cross-validation using a rolling forecasting origin
    "forecast_horizon": 16, # the length of the validation set
    "folds": 1, # the number of training sets (setting to 1 means the standard train-validation split)
    
    # the number of previous days to check for zero sales; if all are zero, generate zero forecasts
    "zero_fc_window": 21,
    
    "static_covs": "keep_all",
    "past_covs": "keep_all",
    "future_covs": "keep_all",
}

In [44]:
class Trainer:
    def __init__(
        self,
        target_dict,
        pipe_dict,
        id_dict,
        past_dict,
        future_dict,
        forecast_horizon,
        folds,
        zero_fc_window,
        static_covs=None,
        past_covs=None,
        future_covs=None,
    ):
        self.target_dict = target_dict.copy()
        self.pipe_dict = pipe_dict.copy()
        self.id_dict = id_dict.copy()
        self.past_dict = past_dict.copy()
        self.future_dict = future_dict.copy()
        self.forecast_horizon = forecast_horizon
        self.folds = int(folds)
        self.zero_fc_window = zero_fc_window
        self.static_covs = static_covs
        self.past_covs = past_covs
        self.future_covs = future_covs
        
    
    def clip(self, array):
        #change negative values to zero
        return np.clip(array, a_min=0., a_max=None)
    
    def train_valid_split(self, target, length):
        #length is (self.folds - j) * self.forecast_horizon 
        train = [t[:-length] for t in target]
        valid_end_idx = -length + self.forecast_horizon
        if valid_end_idx >= 0:
            valid_end_idx = None
        valid = [t[-length:valid_end_idx] for t in target]
        
        return train, valid
    
    def get_models(self, model_names, model_configs):
        models = {
            "lr": LinearRegressionModel,
            "lgbm": LightGBMModel,
            "cat": CatBoostModel,
            "xgb": XGBModel,
        }
        assert isinstance(model_names, list) and isinstance(model_configs, list),\
        "Both the model names and model configurations must be specified in lists."
        assert all(name in models for name in model_names),\
        f"Model names '{model_names}' not recognized."
        assert len(model_names) == len(model_configs),\
        "The number of model names and the number of model configurations do not match."
        
        if "xgb" in model_names:
            xgb_idx = np.where(np.array(model_names)=="xgb")[0]
            for idx in xgb_idx:
                # change to histogram-based method for XGBoost to get faster training time
                model_configs[idx] = {"tree_method": "hist", **model_configs[idx]}
        
        return [models[name](**model_configs[j]) for j, name in enumerate(model_names)]
    
    def generate_forecasts(self, models, train, pipe, past_covs, future_covs, drop_before):
        if drop_before is not None: 
            date = pd.Timestamp(drop_before) - pd.Timedelta(days=1) 
            #train without specifed dates
            train = [t.drop_before(date) for t in train] 
        #inputs for a model
        inputs = {
            "series": train,
            "past_covariates": past_covs,
            "future_covariates": future_covs,
        }

        #generates validation dates and all zero values
        zero_pred = pd.DataFrame({ 
            "date": pd.date_range(train[0].end_time(), periods=self.forecast_horizon+1)[1:],
            "sales": np.zeros(self.forecast_horizon),
        })
        #transforming that df to time series
        zero_pred = TimeSeries.from_dataframe( 
            df=zero_pred,
            time_col="date",
            value_cols="sales",
        )
        
        pred_list = []
        ens_pred = [0 for _ in range(len(train))] #zero for every store 
        
        for m in models:
            # fit training data to model
            m.fit(**inputs)

            # generate forecasts
            pred = m.predict(n=self.forecast_horizon, **inputs)
            #apply inverse transformations
            pred = pipe.inverse_transform(pred)

            for j in range(len(train)):
                #if there is all zeros in j time series in the last specifed period of time predict zeros
                if train[j][-self.zero_fc_window:].values().sum() == 0:
                    pred[j] = zero_pred
            
            # clip negative forecasts to 0s
            pred = [p.map(self.clip) for p in pred]
            pred_list.append(pred)
            
            # ensemble averaging
            for j in range(len(ens_pred)): #54
                ens_pred[j] += pred[j] / len(models) 

        return pred_list, ens_pred
    
    def metric(self, valid, pred):

        valid_df = pd.concat([ts.pd_dataframe() for ts in valid], axis=1)
        pred_df = pd.concat([ts.pd_dataframe() for ts in pred], axis=1)

        # calculate RMSLE for each pair of valid and predicted values
        rmsle_values = [np.sqrt(mean_squared_log_error(valid_df[col], pred_df[col],squared=False)) for col in valid_df.columns]

        # calculate the mean of RMSLE values of all series of that family
        mean_rmsle = np.mean(rmsle_values)

        return mean_rmsle
    
    def validate(self, model_names, model_configs, drop_before=None):
        # helper value to align printed text below
        longest_len = len(max(self.target_dict.keys(), key=len)) #33 kao broj prodavnica
        
        # store metric values for each model
        model_metrics_history = []
        ens_metric_history = []
        
        for fam in tqdm_notebook(self.target_dict, desc="Performing validation"):
            target = self.target_dict[fam]
            pipe = self.pipe_dict[fam]
            past_covs = self.past_dict[fam]
            future_covs = self.future_dict[fam]
            
            # record average metric value over all folds
            model_metrics = []
            ens_metric = 0
            
            for j in range(self.folds):    #folds=1
                # perform train-validation split and apply transformations
                length = (self.folds - j) * self.forecast_horizon #16
                train, valid = self.train_valid_split(target, length) 
                valid = pipe.inverse_transform(valid) 

                # generate forecasts and compute metric
                models = self.get_models(model_names, model_configs)
                pred_list, ens_pred = self.generate_forecasts(models, train, pipe, past_covs, future_covs, drop_before) ################################################################
                metric_list = [self.metric(valid, pred) / self.folds for pred in pred_list]
                model_metrics.append(metric_list)
                if len(models) > 1:
                    ens_metric_fold = self.metric(valid, ens_pred) / self.folds
                    ens_metric += ens_metric_fold
                
            # store final metric value for each model
            model_metrics = np.sum(model_metrics, axis=0)
            model_metrics_history.append(model_metrics)
            ens_metric_history.append(ens_metric)
            
            # print metric value for each family
            print(
                fam,
                " " * (longest_len - len(fam)),
                " | ",
                " - ".join([f"{model}: {metric:.5f}" for model, metric in zip(model_names, model_metrics)]),
                f" - ens: {ens_metric:.5f}" if len(models) > 1 else "",
                sep="",
            )
            
        # print overall metric value
        print(
            "Average RMSLE | "
            + " - ".join([f"{model}: {metric:.5f}" 
                          for model, metric in zip(model_names, np.mean(model_metrics_history, axis=0))])
            + (f" - ens: {np.mean(ens_metric_history):.5f}" if len(models) > 1 else ""),
        )
        
    def ensemble_predict(self, model_names, model_configs, drop_before=None):        
        forecasts = []
        for fam in tqdm_notebook(self.target_dict.keys(), desc="Generating forecasts"):
            target = self.target_dict[fam]
            pipe = self.pipe_dict[fam]
            target_id = self.id_dict[fam]
            past_covs = self.past_dict[fam]
            future_covs = self.future_dict[fam]
            
            models = self.get_models(model_names, model_configs)
            pred_list, ens_pred = self.generate_forecasts(models, target, pipe, past_covs, future_covs, drop_before)
            ens_pred = [p.pd_dataframe().assign(store_number=i['store_nbr']['sales'], family=i['family']) for p, i in zip(ens_pred, target_id)]
            ens_pred = pd.concat(ens_pred, axis=0)
            forecasts.append(ens_pred)
            
        # combine all forecasts into one dataframe
        forecasts = pd.concat(forecasts, axis=0)
        forecasts = forecasts.rename_axis(None, axis=1).reset_index(names="date")
        
        return forecasts

In [45]:
trainer = Trainer(**TRAINER_CONFIG)

In [46]:
BASE_CONFIG = {
    "random_state": 0,
    
    # the number of lag values of the target series
    "lags": 63,
    
    # the number of lag values of the past covariates
    "lags_past_covariates": list(range(-16, -23, -1)),
    
    # the number of (past, future-1) lag values of the future covariates
    "lags_future_covariates": (14, 1),
    
    # the number of days ahead that the model is forecasting given today's input data
    "output_chunk_length": 1,
}

In [47]:
trainer.validate(["lr"], [BASE_CONFIG], drop_before="2014-06-01")

Performing validation:   0%|          | 0/33 [00:00<?, ?it/s]

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.


AUTOMOTIVE                 | lr: 0.70056


`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.


BABY CARE                  | lr: 0.42990


...

PRODUCE                    | lr: 0.59261


`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.


SCHOOL AND OFFICE SUPPLIES | lr: 1.00870


`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.


SEAFOOD                    | lr: 0.66672
Average RMSLE | lr: 0.57994


In [48]:
predictions = trainer.ensemble_predict(
    model_names=["lr"], 
    model_configs=[BASE_CONFIG],
    drop_before="2014-06-01",
)

Generating forecasts:   0%|          | 0/33 [00:00<?, ?it/s]

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
`predict()` was called with `n > output_

In [49]:
final_predictions = predictions.copy()
final_predictions['store_number']=final_predictions['store_number'].astype(int)
final_predictions.rename(columns={'store_number': 'store_nbr'}, inplace=True)
final_predictions.tail()

,date,sales,store_nbr,family
28507,2017-08-27,0.166696,54,SEAFOOD
28508,2017-08-28,0.000000,54,SEAFOOD
28509,2017-08-29,0.302703,54,SEAFOOD
28510,2017-08-30,0.075346,54,SEAFOOD
28511,2017-08-31,0.149236,54,SEAFOOD


In [50]:
test=data[data['date']>= '2017-08-16']

In [51]:
test=test.copy()
start_id = 3000888
end_id = start_id + len(test)
ids = range(start_id, end_id)

test['id'] = ids
test.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,lag_16_sales,lag_17_sales,lag_18_sales,lag_19_sales,lag_20_sales,lag_30_sales,lag_365_sales,lag_730_sales,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,date_index,season,workday,wageday,day_to_nearest_holiday,day_from_nearest_holiday,lag_1_oil,lag_2_oil,lag_3_oil,lag_4_oil,id
3008016,2017-08-16,1,AUTOMOTIVE,NaN,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.8,0.0,8.0,1.0,4.0,7.0,5.0,2.0,5.0,0.0,8,16,228,3,33,3,2017,0,3,0,0,0,0,0,0,1688,2,1,0,5,54,47.57,47.59,47.996667,48.403333,3000888
3008017,2017-08-16,1,BABY CARE,NaN,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,16,228,3,33,3,2017,0,3,0,0,0,0,0,0,1688,2,1,0,5,54,47.57,47.59,47.996667,48.403333,3000889
3008018,2017-08-16,1,BEAUTY,NaN,2.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.8,0.0,3.0,2.0,3.0,2.0,1.0,5.0,5.0,2.0,8,16,228,3,33,3,2017,0,3,0,0,0,0,0,0,1688,2,1,0,5,54,47.57,47.59,47.996667,48.403333,3000890
3008019,2017-08-16,1,BEVERAGES,NaN,20.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.8,0.0,2414.0,1212.0,2161.0,2358.0,2002.0,2381.0,2010.0,2194.0,8,16,228,3,33,3,2017,0,3,0,0,0,0,0,0,1688,2,1,0,5,54,47.57,47.59,47.996667,48.403333,3000891
3008020,2017-08-16,1,BOOKS,NaN,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.8,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8,16,228,3,33,3,2017,0,3,0,0,0,0,0,0,1688,2,1,0,5,54,47.57,47.59,47.996667,48.403333,3000892


In [52]:
final = final_predictions.copy()
submission = test.merge(
    final, on=["date", "store_nbr", "family"], how="left",
)
submission=submission.reset_index()
submission=submission[['id','sales_y']]
submission.rename(columns={'sales_y': 'sales'}, inplace=True)
submission.head()

,id,sales
0,3000888,0.786582
1,3000889,0.000000
2,3000890,1.110778
3,3000891,262.677826
4,3000892,0.000000


In [53]:
#submission.to_csv("submission2.csv", index=False)